In [1]:
import sys
import os

BASE_PATH = os.getenv('PROJECTPATH')

# Add the src directory to the system path
src_path = os.path.join(BASE_PATH, 'src')
sys.path.append(src_path)

In [2]:
import great_expectations as gx
import pandas as pd
import numpy as np
import os
from data import preprocess_data, read_datastore

/home/user/project/MLOps-for-music-popularity-prediction/.venv/lib/python3.11/site-packages/passlib/utils/__init__.py:854: DeprecationWarning: 'crypt' is deprecated and slated for removal in Python 3.13
  from crypt import crypt as _crypt



In [3]:
df, version = read_datastore()
X, y = preprocess_data(df)
X.head()

collecting all words and their counts
PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
PROGRESS: at sentence #10000, processed 45379 words, keeping 1570 word types
PROGRESS: at sentence #20000, processed 90266 words, keeping 1843 word types
PROGRESS: at sentence #30000, processed 135406 words, keeping 2025 word types
PROGRESS: at sentence #40000, processed 179649 words, keeping 2120 word types
PROGRESS: at sentence #50000, processed 223510 words, keeping 2196 word types
PROGRESS: at sentence #60000, processed 267244 words, keeping 2263 word types
PROGRESS: at sentence #70000, processed 310947 words, keeping 2333 word types
PROGRESS: at sentence #80000, processed 354390 words, keeping 2374 word types
PROGRESS: at sentence #90000, processed 398221 words, keeping 2408 word types
PROGRESS: at sentence #100000, processed 442213 words, keeping 2445 word types
PROGRESS: at sentence #110000, processed 485890 words, keeping 2479 word types
PROGRESS: at sentence #120000, process

,multilabel__AD,multilabel__AE,multilabel__AG,multilabel__AL,multilabel__AM,multilabel__AO,multilabel__AR,multilabel__AT,multilabel__AU,multilabel__AZ,...,n_vec_0,n_vec_1,n_vec_2,n_vec_3,n_vec_4,n_vec_5,n_vec_6,n_vec_7,n_vec_8,n_vec_9
0,False,False,False,False,False,False,False,False,False,False,...,-0.052142,-2.310672,1.108316,0.077332,-0.054487,0.754147,4.926912,1.794598,-0.311319,-1.024388
1,True,True,True,True,True,True,True,True,True,True,...,-1.257031,-3.703497,0.204217,2.527353,1.674179,-0.486337,4.744979,3.228882,-0.926144,0.254803
2,False,False,False,False,False,False,False,False,False,False,...,-1.749246,-2.595474,-0.011815,2.139168,3.275259,-0.724204,3.221326,1.753601,-4.204691,-2.353610
3,False,False,False,False,False,False,False,False,False,False,...,1.625497,-0.041652,2.273949,-0.374084,0.649854,1.729393,-0.094518,1.304834,-2.331334,0.270783
4,True,True,True,True,True,True,True,True,True,True,...,0.871466,-0.598964,0.671752,-1.125874,0.811582,0.358382,1.715948,1.591584,-3.742993,-0.473966


In [4]:
import great_expectations as gx
from great_expectations.data_context import FileDataContext
context = gx.get_context(context_root_dir=f"{BASE_PATH}/services/gx")
context

FileDataContext loading fluent config
Loading 'datasources' ->
[{'assets': [...], 'name': 'sample', 'type': 'pandas'},
 {'assets': [...], 'name': 'sample_data', 'type': 'pandas'},
 {'assets': [...], 'name': 'features', 'type': 'pandas'}]
_PandasDataAsset.dict() - missing config_provider, skipping config substitution
_PandasDataAsset.dict() - missing config_provider, skipping config substitution
_PandasDataAsset.dict() - missing config_provider, skipping config substitution
Loaded 'sample' from fluent config
Loaded 'sample_data' from fluent config
Loaded 'features' from fluent config


{
  "anonymous_usage_statistics": {
    "explicit_id": true,
    "usage_statistics_url": "https://stats.greatexpectations.io/great_expectations/v1/usage_statistics",
    "enabled": true,
    "explicit_url": false,
    "data_context_id": "658ab671-7357-4b71-8760-718815eecf13"
  },
  "checkpoint_store_name": "checkpoint_store",
  "config_variables_file_path": "uncommitted/config_variables.yml",
  "config_version": 3.0,
  "data_docs_sites": {
    "local_site": {
      "class_name": "SiteBuilder",
      "show_how_to_buttons": true,
      "store_backend": {
        "class_name": "TupleFilesystemStoreBackend",
        "base_directory": "uncommitted/data_docs/local_site/"
      },
      "site_index_builder": {
        "class_name": "DefaultSiteIndexBuilder"
      }
    }
  },
  "datasources": {},
  "evaluation_parameter_store_name": "evaluation_parameter_store",
  "expectations_store_name": "expectations_store",
  "fluent_datasources": {
    "sample": {
      "type": "pandas",
      "assets":

In [5]:
data_asset = context.get_datasource("features").get_asset("features_dataframe")
batch_request = data_asset.build_batch_request(dataframe=X)

context.add_or_update_expectation_suite("features_expectations")

context.list_expectation_suites()

[ExpectationSuiteIdentifier::features_expectations,
 ExpectationSuiteIdentifier::expectation_suite]

In [6]:
for i in X.columns:
    print(i, X[i].nunique())

multilabel__AD 2
multilabel__AE 2
multilabel__AG 2
multilabel__AL 2
multilabel__AM 2
multilabel__AO 2
multilabel__AR 2
multilabel__AT 2
multilabel__AU 2
multilabel__AZ 2
multilabel__BA 2
multilabel__BB 2
multilabel__BD 2
multilabel__BE 2
multilabel__BF 2
multilabel__BG 2
multilabel__BH 2
multilabel__BI 2
multilabel__BJ 2
multilabel__BN 2
multilabel__BO 2
multilabel__BR 2
multilabel__BS 2
multilabel__BT 2
multilabel__BW 2
multilabel__BY 2
multilabel__BZ 2
multilabel__CA 2
multilabel__CD 2
multilabel__CG 2
multilabel__CH 2
multilabel__CI 2
multilabel__CL 2
multilabel__CM 2
multilabel__CO 2
multilabel__CR 2
multilabel__CV 2
multilabel__CW 2
multilabel__CY 2
multilabel__CZ 2
multilabel__DE 2
multilabel__DJ 2
multilabel__DK 2
multilabel__DM 2
multilabel__DO 2
multilabel__DZ 2
multilabel__EC 2
multilabel__EE 2
multilabel__EG 2
multilabel__ES 2
multilabel__ET 2
multilabel__FI 2
multilabel__FJ 2
multilabel__FM 2
multilabel__FR 2
multilabel__GA 2
multilabel__GB 2
multilabel__GD 2
multilabel__GE

In [7]:
validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name="features_expectations"
)


for column in X.columns:
    # All columns should not have null values
    validator.expect_column_values_to_not_be_null(column)
    if column.startswith("multilabel") or column.startswith("categorical") or column.startswith("bool"):
        # Expect the column to be boolean (True or False)
        validator.expect_column_values_to_be_in_set(column, [True, False])

    elif column.endswith("_sin") or column.endswith("_cos"):
        validator.expect_column_values_to_be_between(column, min_value=-1, max_value=1)
        validator.expect_column_values_to_be_of_type(column, "float")

    elif column.endswith("year") or column.endswith("_length") or column.endswith("_count") or column.startswith("int"):
        # All columns should be positive
        validator.expect_column_min_to_be_between(column, min_value=0, max_value=None)
        # Column must be of type int
        validator.expect_column_values_to_be_of_type(column, "int")

    elif column.startswith("normal") or column.startswith("uniform") or column.startswith("g_vec"):
        # Column must be of type float
        validator.expect_column_values_to_be_of_type(column, "float")

/home/user/project/MLOps-for-music-popularity-prediction/.venv/lib/python3.11/site-packages/great_expectations/expectations/expectation.py:1518: UserWarning: result_format configured at the Validator-level will not be persisted. Please add the configuration to your Checkpoint config or checkpoint_run() method instead.
  warnings.warn(



Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

...

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

In [8]:
validator.save_expectation_suite(discard_failed_expectations=False)

	487 expectation(s) included in expectation_suite.


In [9]:
checkpoint = context.add_or_update_checkpoint(
    name = "features_test_checkpoint",
    validations = [
        {
            "batch_request": batch.batch_request,
            "expectation_suite_name": "features_expectations"
        } for batch in data_asset.get_batch_list_from_batch_request(batch_request)
    ]
)

In [11]:
# Print passed expectations
res = checkpoint.run()
res

	487 expectation(s) included in expectation_suite.


Calculating Metrics:   0%|          | 0/2271 [00:00<?, ?it/s]

{
  "run_id": {
    "run_name": null,
    "run_time": "2024-07-23T08:53:19.521387+03:00"
  },
  "run_results": {
    "ValidationResultIdentifier::features_expectations/__none__/20240723T055319.521387Z/features-features_dataframe": {
      "validation_result": {
        "success": true,
        "results": [
          {
            "success": true,
            "expectation_config": {
              "expectation_type": "expect_column_values_to_not_be_null",
              "kwargs": {
                "column": "multilabel__AD",
                "batch_id": "features-features_dataframe"
              },
              "meta": {}
            },
            "result": {
              "element_count": 179940,
              "unexpected_count": 0,
              "unexpected_percent": 0.0,
              "partial_unexpected_list": [],
              "partial_unexpected_counts": [],
              "partial_unexpected_index_list": []
            },
            "meta": {},
            "exception_info": {
   